<a href="https://colab.research.google.com/github/Muhammad-Abdullah-Jami/BERT-LLM/blob/main/Bert_house_sale_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  !pip install transformers datasets scikit-learn


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import Trainer, TrainingArguments

In [8]:

# Load the Ames Housing dataset
url = "https://raw.githubusercontent.com/Muhammad-Abdullah-Jami/BERT-LLM/main/AmesHousing.csv"
df = pd.read_csv(url)

# Display the first few rows of the dataset
print(df.head())

# Use 'Neighborhood' as a description and 'SalePrice' as the target variable
# Drop rows with missing values in the selected columns
df = df.dropna(subset=['Neighborhood', 'SalePrice'])

# Keep only the relevant columns
df = df[['Neighborhood', 'SalePrice']]

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Display the sizes of the training and validation sets
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")


   Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      1  526301100           20        RL         141.0     31770   Pave   
1      2  526350040           20        RH          80.0     11622   Pave   
2      3  526351010           20        RL          81.0     14267   Pave   
3      4  526353030           20        RL          93.0     11160   Pave   
4      5  527105010           60        RL          74.0     13830   Pave   

  Alley Lot Shape Land Contour  ... Pool Area Pool QC  Fence Misc Feature  \
0   NaN       IR1          Lvl  ...         0     NaN    NaN          NaN   
1   NaN       Reg          Lvl  ...         0     NaN  MnPrv          NaN   
2   NaN       IR1          Lvl  ...         0     NaN    NaN         Gar2   
3   NaN       Reg          Lvl  ...         0     NaN    NaN          NaN   
4   NaN       IR1          Lvl  ...         0     NaN  MnPrv          NaN   

  Misc Val Mo Sold Yr Sold Sale Type  Sale Condition  SalePrice  
0       

In [9]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to tokenize the text data
def tokenize_data(df):
    return tokenizer(
        df['Neighborhood'].tolist(),  # Change from 'description' to 'Neighborhood'
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

# Tokenize training and validation data
train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:

# Convert labels to tensor format
train_labels = torch.tensor(train_df['SalePrice'].values, dtype=torch.float32).view(-1, 1)  # Changed 'price' to 'SalePrice'
val_labels = torch.tensor(val_df['SalePrice'].values, dtype=torch.float32).view(-1, 1)  # Changed 'price' to 'SalePrice'

# Create a PyTorch Dataset class
class HouseSalesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create the datasets
train_dataset = HouseSalesDataset(train_encodings, train_labels)
val_dataset = HouseSalesDataset(val_encodings, val_labels)

In [11]:
# Load the BERT model for regression (we treat it as a regression problem)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Define a compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids.flatten()
    preds = pred.predictions.flatten()
    mse = mean_squared_error(labels, preds)
    return {'mse': mse, 'rmse': mse ** 0.5}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


NameError: name 'mean_squared_error' is not defined

In [ ]:
# Evaluate the model on the validation set
trainer.evaluate()


In [ ]:
# Function to generate predictions
def predict(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    inputs = inputs.to(model.device)  # Move to GPU if available
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = outputs.logits.flatten()
    return predictions.cpu().numpy()

# Example predictions
sample_descriptions = [
    "Beautiful home with a large garden.",
    "Compact studio apartment in the city center."
]
predictions = predict(sample_descriptions)
print(predictions)  # Output: predicted prices
